<h2>ACTF</h2> 

<h>SECURE CONNECTION</h>

<p>Description:</p><p>We leak some packets log in author’s PC and get part of the secureconn software, can you get the flag? (software is buggy, don`t mind it and just get your flag)</p>
<p>Attachments:</p><p>The big client.py and core.py files are in directory. Also there's a log file master.txt, that contains:</p>

<p>First of all, I've written a parser for this dump file</p><p>It was not hard, since all the instructions for decomposing were in core.py file</p><p>Here's the result:</p>

<p>You will not find any valuable information in client.py</p>
<p>Now we can draw a few conclusions, based on dump file and core.py file</p>
<p>Obviously the first of them is master and the second is slave</p>
<p>First 7 messages are not interesting. All we get from them is that they are using a secure connection from now on</p>
<p>Messages from 8 to 15 are a handshake between sockets. From which we can get:</p>
<p>8-9: crc seed 0xd9b2df to check that connection was not interrupted by anyone</p>
<p>10: master's IV = ec36e5b06955d995 and Secret = 567ee5de450737f8</p>
<p>11: slave's  IV = 68b3ded5b84014dc and Secret = f3fb7502d9390e34</p>
<p>12: master's confirm = 9f5136cacd9f2a5387394b7d0c1cXXXX (note that we don't know the last two bytes)</p>
<p>13: slave's  confirm = XXd6e4XXXX5cXXb7ba906e57055a8ec8 (now we don't know 4 bytes)</p>
<p>14: master's random_value = 4bd20924f0c3cd30ba64a0f1d964691e</p>
<p>15: slave's  random_value = dd76514f5736813aa8c2178eXXf82d5b(1 byte unknown)</p>
<p>Then they send data to each other in which we are not interested now</p> 

In [41]:
from Crypto.Cipher import AES
import base64
import libscrc
from multiprocessing import Pool
from time import time
from Crypto.Util.number import long_to_bytes
from copy import copy

<p>The key to the solution is the fact that their shared key is very small(it's mod(0x1000000) hence we can iterate over this key</p>

In [81]:
def calc_crc(crc, pdu):
    initvalue = int.from_bytes(crc, "little")
    crc = libscrc.hacker24(data=pdu, poly=0x00065B, init=initvalue,
                            xorout=0x00000000, refin=True, refout=True)
    return crc.to_bytes(3, "little")

def bytes_xor_16(bytes1, bytes2):
    v1 = int.from_bytes(bytes1, 'big')
    v2 = int.from_bytes(bytes2, 'big')
    v3 = v1 ^ v2
    return (v3).to_bytes(16, 'big')

def secure_decrypt_packet(key, plain, nonce):
    aes = AES.new(key=key, mode=AES.MODE_CCM, nonce=nonce)
    return aes.decrypt(plain)

def secure_encrypt(key, plain):
    aes = AES.new(key=key, mode=AES.MODE_ECB)
    return aes.encrypt(plain)

def secure_confirm(key, r, p1, p2):
    return secure_encrypt(key, bytes_xor_16(secure_encrypt(key, bytes_xor_16(r, p1)), p2))

In [82]:
crc_seed = bytes.fromhex('d9b2df')

m_IV =     bytes.fromhex('ec36e5b06955d995')
m_Secret = bytes.fromhex('567ee5de450737f8')
s_IV =     bytes.fromhex('68b3ded5b84014dc')
s_Secret = bytes.fromhex('f3fb7502d9390e34')
m_random = bytes.fromhex('4bd20924f0c3cd30ba64a0f1d964691e')

<p>Now let's find out what are the XX unknown values using simple iteration</p>

In [83]:
def get_m_confirm(crc_seed, crc):
    for i in range(256):
        for j in range(256):
            x, y = hex(i)[2:].zfill(2), hex(j)[2:].zfill(2)
            s = bytes.fromhex(f'84109f5136cacd9f2a5387394b7d0c1c{x}{y}')  # note that we use the whole message, 
                                                                          # including first two bytes 0x84 and 0x10 + data
            if(crc == calc_crc(crc_seed, s)):
                return s
m_confirm_crc = bytes.fromhex('584605')
m_confirm = get_m_confirm(crc_seed, m_confirm_crc)[2:]
print(m_confirm)

b'\x9fQ6\xca\xcd\x9f*S\x879K}\x0c\x1c\x16\xfa'


<p>We know that m_confirm = aes(key=shared_key, mode=CCM, plain=plain)</p>
<p>where plain = aes(key=shared_key, mode=ECB, plain=plain1)</p>
<p>plain1 = b'\xff' * 16 <b>XOR</b> aes(key = shared_key, mode=ECB, plain=m_random <b>XOR</b> 0)</p>
<p>Hence we can iterete over [0, 0x1000000] to find the key</p>

In [33]:
def get_key(g):
    for x in g:
        x = x.to_bytes(16, "little")
        if secure_confirm(x, m_random, b"\x00"*16, b"\xff"*16) == m_confirm:
            print(f"shared key = {x}")
gs = [range(i, 256**3, 32) for i in range(32)]
with Pool(10) as pool:
    pool.map(get_key, gs)

shared key = b'%=\x8c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [84]:
shared_key = b'%=\x8c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
assert m_confirm == secure_confirm(shared_key, m_random, b"\x00"*16, b"\xff"*16)

<p>The rest is quite clear: we have to find s_random to calculate:</p>
<p>storekey = secure_encrypt(numeric_key, m_random[:8] + s_random[8:])</p>
<p>And then</p>
<p>sessionkey = secure_encrypt(storekey, m_secret + s_secret)

<p>Finding s_random is very easy, since we have already done it with m_confrim.</p>

In [85]:
def get_s_random(crc_seed, crc):
    for i in range(256):
        x = hex(i)[2:].zfill(2)
        s = bytes.fromhex(f'8710dd76514f5736813aa8c2178e{x}f82d5b')  # note that we use the whole message, 
                                                                     # including first two bytes 0x87 and 0x10 + data
        if(crc == calc_crc(crc_seed, s)):
            return s
s_random_crc = bytes.fromhex('6f68ec')
s_random = get_s_random(crc_seed, s_random_crc)[2:]

<p>We could also calculate s_confirm, but this is only necessary for verification, and there are many values that correspond to crc = '2ddbb8'.It is not surprising, since we have to iterate over 3 unknown bytes.</p>

In [93]:
store_key = secure_encrypt(shared_key, m_random[:8] + s_random[8:])
sessionkey = secure_encrypt(store_key, m_Secret + s_Secret)

In [94]:
known_ciphertexts = [
    "ee491a84624116fb685e5d471494aa6d3eac7c53707c465050907ea20112040690025e92a61dd8291b50d0c16913b9cd0ff5290edad9c23d69384649765b847f15f221ce",
    "ea4d61864a515fe478413b4c1294b57a388207145b56224a50916abe01121f1280106fc5a577a83a1d40af897a07a18d0cdf1318f2d2d27e424c55575c20907d2df2478a0519c8170633f1a94db615ac37bba648c133dff426c20a28f9125fe1fd35d0af550701851692626b6ffac7434f92b568c266533652de21864323033898f514fd5cb0ef2059fe9ab68e2917d75d5ccfc6a8c21dba69d73bb79944c38bb5208ffe67e028649a406a2bd71d8670f19fefa719cfdbe672f4c58a1e2d1c092c3f21db23bf63f7da5d78905602f222e458a5ca7a04835d4cd90a1a5d900a78f67516ea443289971a7fe2da157d60ce1b6331acc87ef69ce9589efa9c5469",
    "8411de79f3a0cfb304f6dfec305c00ca30d769829e559b428dc6f0ae6d8b73d9afbbbfa8b4f4e5ad6bbe553beb3497882b8a413feee320f63869b79b98ac6a6783e0e5dee5e18e804313e22e56383afdb4eaa54487ad8aec5a5e016e5ddb3944813957e70524e058e85641fa4dcdb2714d6aa479160b4368c8dbadd66d8d8a9e4c8a7f584554f31522823559381e754e8cc8c6a00be26d750d7849366eccb224909dc98bda4e5181153c6707c0f65c9c6da1148cfefdc77a65636917f93c8c0d447ebd7e49894fb4617ab6b3709e2ab3b9c9fe18947eb45085e7b9e72cdbc01092ac603cc2f7cbfbfbb69ff9affaba609b99cf35694b9b9ef4cab3dfbc1d7b",
    "4a21065d5ab2a0e2cb4f31e22bddd9576e81cd3105dc91a9fb9db0dcec197be84e441a79ecb41553852f1558785dc31f036208a452c357b1524cf56dbcdf985e6435b8f6174cfd28d92e3d30abe982ee10d80a753155bed89c85bad3649bed2f2e41a53c1a1edd6547227014868235ac5ebbe6e8c7cb92640d0cdd81a69135ad3b3639bee246285cc513cb6d216447342c596d77dfe64a06667b64f4b75ac7c603cb5c02aceaf4f780ec1cc43fed5fb8cf194b029d8e485fff93695f37862102b76060549ea9d0c5f852be7ced74e30dcda4bb9513a957fae08e41aa0974b5b04567f8a49da94c0fc8f2820a457118daece75a4ed45d0db8757c47a9d185e5",
    "a6367b6aa555af69a9a97d0e09aa4886d52720c77465e33718768d1489d9d1cc84d0ed7bd60455002e04ee7fae368c478382a2ef264bdd9173d28c29315b8f3e3c19248950bed65fe788e4ac137126851bc88d4794e641859e6fb2",
    "b729d427d4a9d5952ec3cecc1e70159c27c6638d8a03ed6cf1e4f5b143961ed9a79faee890f5ecad639e4f09ce13cfbc33d84f27c8ea3ace1178a8b18e9f6b5face2e8ebedc48fae7a36d500600a53ea89e8c61a95c5fcd85445711563fe1664d12142ee112af26dcb7340a345d0996a4952b13f1f703d4c99b1b3e902878ff745ac61216b49d838058d0a6837001b11bcc6c48231eb51445f74483558ddbc119ff7b985cb1e69b00b424875e2d04d9665f20185e997bc4872474254e12d99547659b75852ba5e994164b7cf459049f280ffff1db370bd7290edb3c537d6a735fa993e09e8c5debcd5858a98f8f4aa4dc9cece013d6f958fdad787e0993644",
    "1013375c5ca983e3905a58f705de88337fb3fc341cdaab9eaecf90ab8b",
    "18aee95ecac09ee63dd28707b8942d4f2a7052d71bfd27d81bcceffd208a1463f9a135248def5781",
    "a2162539df5bac459586535812db74a6cb541dd71f64ec4d12719f32a6def899e3d7eb62c4127702173ec242bc32aa5e82fee8ea335bc4ad7dc8f22e2059a30419171abe73afe65bfaa6ada32a15788d0db6b359b0be7fa6af68cde6e24ca95d",
    "fd81d2b58c5e3206"]

<p>Well, you can just count the number of messages sent by each side or iterate over [0, 10] to get a one-time number for aes encryption.</p>

In [95]:
for cip in known_ciphertexts:
    for n in range(10):
        try:
            pl = base64.b64decode(secure_decrypt_packet(sessionkey, bytes.fromhex(cip), n.to_bytes(13, "little")) + b'====')
            if(pl.decode()):
                print(pl, n)
        except Exception as e:
            continue

b'I will tell you my flag after you finish your poem' 4
b"You mean this one? Shall I compare thee to a summer's day? Thou art more lovely and more temperate: Rough winds do shake the darling buds of May, And summer's lease hath all too short a date:" 4
b'No I mean this one, I never saw a Moor-I never saw the Sea-Yet know I how the Heather looksAnd what a Billow be.I never spoke with GodNor visited in Heaven-Yet certain am I of the spotAs if t' 5
b'q;cM8' 1
b'Nevermind, long live the AAA' 8
b'You got your flag: ACTF{ShORt_NUmeR1c_KEY_1s_Vuln3R4bLe_TO_e@V3sDropPEr}' 7
b'\x07' 7
b'\x06' 8
b'Cool' 9
